In [1]:
from __future__ import print_function
from six.moves import range
from PIL import Image

import sys
# dir_path = '~/GANtor-Arts-Center/src/code/main.py'
# sys.path.append(dir_path)
sys.path.append('../')

import torch.backends.cudnn as cudnn
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import os
import time

import numpy as np
import torchfile

from miscc.config import cfg, cfg_from_file
from miscc.utils import mkdir_p
from miscc.utils import weights_init
from miscc.utils import save_img_results, save_model
from miscc.utils import KL_loss
from miscc.utils import compute_discriminator_loss, compute_generator_loss

from tensorboard import summary
from tensorboardX import FileWriter

import torchvision.utils as vutils


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device(0)
print(torch.cuda.is_available())

True


In [ ]:
from model import STAGE1_G, STAGE2_G, STAGE2_D


stage = 1
category = "style"
image_size = 64 if stage == 1 else 256
image_dir = './baseline_generated/{}{}/'.format(category, image_size)

style_1 = '../../../results/wikiart_stageI_2019_05_14_23_03_43/Model/netG_epoch_60.pth'
style_2 = '../../../results/wikiart_stageII_2019_05_15_03_54_54/Model/netG_epoch_45.pth'

genre_1 = "../../saved_models/stage1_genre/netG_epoch_70.pth"
genre_2 = ""

config_file = '../cfg/wikiart_s2_style.yml' if category == "style" else '../cfg/wikiart_s2.yml'
cfg_from_file(config_file)

Stage1_G = STAGE1_G()

stage_1_file = genre_1 if category == "genre" else style_1
stage_2_file = genre_2 if category == "genre" else style_2

if stage == 1:
    netG = Stage1_G
    state_dict = torch.load(stage_1_file,map_location=lambda storage, loc: storage)
    netG.load_state_dict(state_dict)
    print('Load from: ', stage_1_file)
    
elif stage == 2:
    netG = STAGE2_G(Stage1_G)
    state_dict = torch.load(stage_1_file,map_location=lambda storage, loc: storage)
    netG.STAGE1_G.load_state_dict(state_dict)
    print('Load from: ', stage_1_file)
    state_dict = torch.load(stage_2_file, map_location=lambda storage, loc: storage)
    netG.load_state_dict(state_dict)
    print('Load from: ', stage_2_file)

else:
    raise Exception ("Stage unspecified!")
    
netG.eval()

if cfg.CUDA:
    netG.cuda()

print(netG)


In [ ]:
nz = 100
batch_size = 16
embedding_size = 27 if category == "style" else 10
num_iters = 100 if category == "style" else 300 #43.2k images for style, 48k for genre

for i in range(num_iters):
    print ("Iter num %i"%(i))
    for class_idx in range(embedding_size):
        noise = Variable(torch.FloatTensor(batch_size, nz))
        with torch.no_grad():
                    fixed_noise = \
                        Variable(torch.FloatTensor(batch_size, nz).normal_(0, 1))
        noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

        noise.data.normal_(0, 1)
        text_embeddings = torch.zeros((batch_size, embedding_size)).cuda()
        text_embeddings[:, class_idx] = 1

#         inputs = (text_embeddings, noise)

        lr_fake, fake = netG(text_embeddings, noise)

        for im_idx, im in enumerate(fake.data):
            
            vutils.save_image(
                        im, '%s%i/%i_%i.png' %
                        (image_dir, class_idx, i, im_idx), normalize=True)